In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('order_brush_order.csv')

In [3]:
df.head()
res = dict()
# res shape should be 18770 * 2
remain_answer_shop = 18770

## Get unique shop id 

In [4]:
print(df.shape)
print("number of shop id: ", df['shopid'].nunique())
print("number of user id: ", df['userid'].nunique())
remain_answer_shop = df['shopid'].nunique()
unique_shopId = df['shopid'].unique()
unique_userId = df['userid'].unique()
print("unique shop id: ", unique_shopId, len(unique_shopId))
print("unique user id: ", unique_userId, len(unique_userId))

(222750, 4)
number of shop id:  18770
number of user id:  93883
unique shop id:  [ 93950878 156423439 173699291 ...  50236030 203587596 147118226] 18770
unique user id:  [30530270 46057927 67341739 ... 55114470 41654255 27306783] 93883


## Filtered by buyers less than 3

In [5]:
group_shop = df.groupby(['shopid'])
filter_shopId = []
for i, shopId in enumerate(unique_shopId):
    if group_shop.get_group(shopId).shape[0] < 3:
        res[shopId] = 0
        filter_shopId.append(shopId)
unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  9739 remain answers:  9031 True


## Filter each shop by count number of each userid which less than 2

In [6]:
# sort event time in df
freqCount = 0
filter_shopId = []
for i, uid in enumerate(unique_shopId):
    testdf = group_shop.get_group(uid).sort_values(["event_time"])
    userfreq = testdf['userid'].value_counts()
    if userfreq.max() >= 3:
        freqCount += 1
    else:
        res[uid] = 0
        filter_shopId.append(uid)
unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  17942 remain answers:  828 True


## Get user dataframe from specific shop dataframe

In [7]:
testdf = group_shop.get_group(unique_shopId[0]).sort_values(["event_time"])
userfreq = testdf['userid'].value_counts()
numberCandicate, candicates = userfreq[userfreq >=3].index.shape[0], np.array(userfreq[userfreq >=3].index)

subtestdf = testdf[testdf['userid'] == candicates[0]].sort_values(["event_time"])
print("number of candicate user id: ", numberCandicate)
print("candicate userids: ",candicates)
print(subtestdf)

number of candicate user id:  1
candicate userids:  [110240400]
              orderid     shopid     userid           event_time
43418  31280563745641  173699291  110240400  2019-12-29 09:02:43
30408  31387219757278  173699291  110240400  2019-12-30 14:40:19
49832  31387324931487  173699291  110240400  2019-12-30 14:42:05


### Note: How to calculate datetime

In [8]:
import datetime
from dateutil.parser import parse
now = datetime.datetime.now()
print(now)
a = parse('January 31, 2010')
print(a)
td = datetime.timedelta(weeks=1, days=30, hours=2, minutes=40)
print(td)
t1 = parse(group_shop.get_group(unique_shopId[0]).sort_values(["event_time"]).iloc[0]['event_time'])
t2 = parse(group_shop.get_group(unique_shopId[0]).sort_values(["event_time"]).iloc[1]['event_time'])
oneHour = datetime.timedelta(hours=1)
print(t1)
print(t2)
print(t2-t1)

print(oneHour)
print((t2-t1) < oneHour)

2021-02-26 11:20:57.353621
2010-01-31 00:00:00
37 days, 2:40:00
2019-12-27 00:48:28
2019-12-27 01:00:48
0:12:20
1:00:00
True


## Get candicates which equal to 3 and difference between start and end time >one hour

In [9]:
oneHour = datetime.timedelta(hours=1)
subTables = []
filter_shopId = []
for i, uid in enumerate(unique_shopId):
    testdf = group_shop.get_group(uid)
    userfreq = testdf['userid'].value_counts()
    numberCandicate = userfreq[userfreq >=3].index.shape[0]
    candicates = np.array(userfreq[userfreq >=3].index)
    
    if(numberCandicate == 1):
        subtestdf = testdf[testdf['userid'] == candicates[0]].sort_values(["event_time"])
        if(subtestdf.shape[0] == 3):
            startT = parse(subtestdf.iloc[0]['event_time'])
            endT = parse(subtestdf.iloc[-1]['event_time'])
            if((endT-startT) > oneHour):
                res[uid] = 0
                filter_shopId.append(uid)

    
unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  18255 remain answers:  515 True


## Filter candicates which equal to 3 and difference between start and end time >one hour

### Condition: number of candicate == 0,  

In [10]:
subTables = []
filter_shopId = []
for i, uid in enumerate(unique_shopId):
    testdf = group_shop.get_group(uid)
    userfreq = testdf['userid'].value_counts()
    candicates = np.array(userfreq[userfreq >=3].index)
    filter_candicates = []
    
    for user in candicates:
        subtestdf = testdf[testdf['userid'] == user].sort_values(["event_time"])
        if(subtestdf.shape[0] == 3):
            startT = parse(subtestdf.iloc[0]['event_time'])
            endT = parse(subtestdf.iloc[-1]['event_time'])
            if((endT-startT) > oneHour):
                filter_candicates.append(user)
    candicates = [c for c in candicates if c not in filter_candicates]
    if len(candicates) == 0:
        res[uid] = 0
        filter_shopId.append(uid)
    else:
        subTables.append({'shopid': uid,
                     "candicates": candicates})

unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)

Got number of answers:  18281 remain answers:  489 True


In [11]:
subTables = sorted(subTables, key=lambda k: len(k['candicates'])) 
print(len(subTables))

489


## The column index (start time and end time) is continue, order brushing

In [12]:
filter_shopId = []
count = 0
for s in subTables:
    testdf = group_shop.get_group(s['shopid']).sort_values(["event_time"])
    testdf['time'] = [parse(testdf.iloc[i]['event_time']) for i in range(testdf.shape[0])]
    testdf.reset_index(drop=True, inplace=True)
    if(len(s['candicates']) == 1):
        subtestdf = testdf[testdf['userid'] == s['candicates'][0]].sort_values(["time"])
        startT = subtestdf.iloc[0]['time']
        endT = subtestdf.iloc[-1]['time']
        if ((endT-startT) < oneHour):
            startIdx = testdf[testdf['time'] == startT].index.values[0]
            endIdx = testdf[testdf['time'] == endT].index.values[0]
            if((endIdx - startIdx + 1) == subtestdf.shape[0]):
                res[s['shopid']] = s['candicates'][0]
                filter_shopId.append(s['shopid'])
                count += 1
                

unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)
subTables = [t for t in subTables if t["shopid"] not in filter_shopId]
print(len(subTables))

Got number of answers:  18542 remain answers:  228 True
228


In [13]:
filter_shopId = []
for s in subTables:
    testdf = group_shop.get_group(s['shopid']).sort_values(["event_time"])
    testdf['time'] = [parse(testdf.iloc[i]['event_time']) for i in range(testdf.shape[0])]
    testdf.reset_index(drop=True, inplace=True)
    if(len(s['candicates']) == 1):
        subtestdf = testdf[testdf['userid'] == s['candicates'][0]].sort_values(["time"])
        startT = subtestdf.iloc[0]['time']
        endT = subtestdf.iloc[-1]['time']
        if ((endT-startT) < oneHour):
            startIdx = testdf[testdf['time'] == startT].index.values[0]
            endIdx = testdf[testdf['time'] == endT].index.values[0]
#             print(testdf.loc[startIdx:endIdx][["userid", "time"]])
            subTestdf = testdf.loc[startIdx:endIdx]
            uniqueUser = subTestdf['userid'].nunique()
            numOders = subTestdf.shape[0]
            isOderBrushing = (numOders/uniqueUser) >=3
            if isOderBrushing:
                res[s['shopid']] = s['candicates'][0]
                filter_shopId.append(s['shopid'])
            else:
                res[s['shopid']] = 0
                filter_shopId.append(s['shopid'])
#             print(subtestdf.shape[0], startIdx, endIdx, uniqueUser, numOders,isOderBrushing, numOders/uniqueUser)

unique_shopId = [item for item in unique_shopId if item not in filter_shopId]
print("Got number of answers: ", len(res), "remain answers: ", len(unique_shopId), len(res)+len(unique_shopId) == remain_answer_shop)
subTables = [t for t in subTables if t["shopid"] not in filter_shopId]
print(len(subTables))

Got number of answers:  18577 remain answers:  193 True
193


In [86]:
import time
from sklearn.neighbors.kde import KernelDensity
from scipy.stats import kde
from matplotlib.pyplot import plot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans

maxN = 0
for s in subTables:
    
    testdf = group_shop.get_group(s['shopid']).sort_values(["event_time"])
    testdf['time'] = [parse(testdf.iloc[i]['event_time']) for i in range(testdf.shape[0])]
    testdf.reset_index(drop=True, inplace=True)
    if len(s['candicates']) == 1:
        subtestdf = testdf[testdf['userid'] == s['candicates'][0]].sort_values(["time"])
        
        subtestdf.drop(['orderid', 'shopid', 'userid', 'event_time'], axis = 1, inplace=True)
        
        maxN = subtestdf.shape[0] if maxN < subtestdf.shape[0] else maxN
        if maxN == 23:
            kmeans = KMeans(n_clusters=12)
            
            subtestdf['label'] = kmeans.fit_predict(subtestdf[['time']])
            print(subtestdf)
            assert(0)
        
        
        
        
        
        
        
        
        
    else:
        pass
#         for c in s['candicates']:
#             subtestdf = testdf[testdf['userid'] == c].sort_values(["time"])
#             maxN = subtestdf.shape[0] if maxN < subtestdf.shape[0] else maxN
#     print(len(s['candicates']), maxN, subtestdf.shape[0])
#         if subTestdf.shape[0] > 10:
#             continue
            
#             startT = subtestdf.iloc[0]['time']
#             endT = subtestdf.iloc[-1]['time']

#             subtestdf['ordinal_d'] = subtestdf['time'].apply(lambda t: t.toordinal() )
#             subtestdf['ordinal_s'] = subtestdf['time'].apply(lambda t: t.timestamp() )


#             timeTest = sorted(np.array(subtestdf['ordinal_d']).reshape((-1, 1)))
#             subtestdf.drop(['orderid', 'shopid', 'userid', 'event_time'], axis = 1, inplace=True) 
#             kmeans = KMeans(n_clusters=2)
#             subtestdf['label_1'] = kmeans.fit_predict(subtestdf[['time']])
#             subtestdf['label_2'] = kmeans.fit_predict(subtestdf[['ordinal_d']])
#             subtestdf['label_3'] = kmeans.fit_predict(subtestdf[['ordinal_s']])
#             print(subtestdf)

#             assert(0)
print(maxN)

                  time     ordinal_s  label_1  label_3
3  2019-12-27 11:49:16  1.577447e+09        1        1
4  2019-12-27 11:54:28  1.577448e+09        1        1
5  2019-12-27 12:10:34  1.577449e+09        9        9
6  2019-12-27 13:18:01  1.577453e+09        5        5
30 2019-12-30 16:37:02  1.577724e+09       11       11
31 2019-12-30 16:48:36  1.577725e+09        6        0
32 2019-12-30 16:52:27  1.577725e+09        6        0
33 2019-12-30 16:56:40  1.577725e+09       10       10
34 2019-12-30 16:59:57  1.577725e+09       10       10
35 2019-12-30 17:16:42  1.577726e+09        0        8
36 2019-12-30 17:21:10  1.577726e+09        0        8
37 2019-12-30 17:26:28  1.577727e+09        8        6
38 2019-12-30 17:30:36  1.577727e+09        8        6
39 2019-12-30 17:35:45  1.577727e+09        8        6
50 2019-12-31 10:25:49  1.577788e+09        3        3
53 2019-12-31 15:14:17  1.577805e+09        2        2
54 2019-12-31 15:15:40  1.577805e+09        2        2
55 2019-12

AssertionError: 